<a href="https://colab.research.google.com/github/LastPudding/Text_Mining_LSTM/blob/main/TextMiningCoursework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import spacy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Concatenate, Input
from tensorflow.keras.models import Model
from keras.models import Sequential

# Load spaCy English tokenizer
nlp = spacy.load("en_core_web_sm")

In [4]:
!ls

sample_data


In [5]:
def extract_triplets(file_path):
    triplets = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            if line:  # Check if the line is not empty
                # Split the line into individual triplets using the delimiter "|"
                individual_triplets = line.split('|')
                triplet_list = []
                # Split each individual triplet into entities and relation
                for triplet in individual_triplets:
                    triplet_parts = triplet.strip().split(';')
                    if len(triplet_parts) == 3:
                        subject, related_entity, relation = [part.strip() for part in triplet_parts]
                        triplet_list.append((subject, related_entity, relation))
                triplets.append(triplet_list)
    return triplets

def read_document(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file]
    return lines

# Function to tokenize sentences
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        tokens = [token.text for token in nlp(sentence)]
        tokenized_sentences.append(tokens)
    return tokenized_sentences

def encode_relations(triplets, relation_to_index):
    encoded_triplets = []
    for triplet_list in triplets:
        encoded_triplet_list = []
        for triplet in triplet_list:
            encoded_triplet = [0] * len(relation_to_index)  # Initialize with zeros
            relation = triplet[2]  # Extract the relation from the triplet
            if relation in relation_to_index:
                encoded_triplet[relation_to_index[relation]] = 1  # Set the corresponding index to 1
            encoded_triplet_list.append(encoded_triplet)
        encoded_triplets.append(encoded_triplet_list)
    return encoded_triplets

In [8]:
# Extracting sentences and Triplets
train_triplets, test_triplets = extract_triplets('train.tup'), extract_triplets('test.tup')
train_sentences, test_sentences = read_document('train.sent'), read_document('test.sent')

# Read unique relations from relations.txt
unique_relations = read_document('relations.txt')
num_unique_relations = len(unique_relations)

#Create a mapping between each relation and its index
relation_to_index = {relation: index for index, relation in enumerate(unique_relations)}




In [9]:
# Prepare input data
X_sentences = tokenize_sentences(train_sentences)
X_triplets = encode_relations(train_triplets, relation_to_index)

# Define maximum sequence length
max_seq_length = max(len(tokens) for tokens in X_sentences)

In [31]:
#GPT
def count_tokens(sentences, min_count=None):
    token_counts = {}  # Create an empty dictionary to store tokens and their counts

    # Iterate over each sentence
    for sentence in sentences:
        # Iterate over each token in the sentence
        for token in sentence:
            # Add the token to the dictionary and update the count
            token_counts[token] = token_counts.get(token, 0) + 1

    # Filter tokens by minimum count if specified
    if min_count is not None:
        token_counts = {token: count for token, count in token_counts.items() if count > min_count}

    # Sort the dictionary by value (token counts) in descending order
    token_counts = dict(sorted(token_counts.items(), key=lambda x: x[1], reverse=True))

    return token_counts
len(count_tokens(X_sentences,1))



11902

In [44]:
#https://medium.com/southpigalle/simple-relation-extraction-with-a-bi-lstm-model-part-1-682b670d5e11
X_sentences_padded = pad_sequences(X_triplets, maxlen=max_seq_length, padding='post')

embedding_dim = 100
lstm_units = 64
vocab_size=12000

embedding_size = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=X_sentences_padded.shape[1]))
model.add(Bidirectional(LSTM(128, dropout=0.7, recurrent_dropout=0.7)))
model.add(Dense(len(unique_relations), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1515, 100)         1200000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 256)               234496    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 29)                7453      
                                                                 
Total params: 1441949 (5.50 MB)
Trainable params: 1441949 (5.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [32]:
# Pad sentences to max sequence length
X_sentences_padded = pad_sequences(X_triplets, maxlen=max_seq_length, padding='post')

# Define LSTM model
embedding_dim = 100
lstm_units = 64
vocab_size=12000

sentence_input = Input(shape=(max_seq_length,), dtype='int32')
triplets_input = Input(shape=(None, len(unique_relations)), dtype='int32')

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length)(sentence_input)
lstm_layer = Bidirectional(LSTM(lstm_units))(embedding_layer)

concatenated_input = Concatenate()([lstm_layer, triplets_input])
output_layer = Dense(len(unique_relations), activation='softmax')(concatenated_input)

model = Sequential()
model.add(embedding_layer)
model.add(lstm_layer)
model.add(output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())
# model = Model(inputs=[sentence_input, triplets_input], outputs=output_layer)

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 128), (None, None, 29)]